<h1 align="center">EDA SQL PCRF RESERVATIONS</h1>

<div style="text-align:center">
    <img src="https://i.ibb.co/y8zVBsk/636493827765700000.jpg">
</div>

Now continue with the analysis, now using SQL query tools to find insights. There are some analyses that could be easier with SQL than using common Python tools. Let’s break in.

### IMPORTING LIBRARIES

We’ll import the necessary libraries for the SQL analysis:

In [76]:
import os
import sqlite3
import pandas as pd

In [77]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### DATA COLLECTION 

We can import the dataset that was filtered and used in the previous analysis:

In [78]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(project_dir, "data", "raw")
path = os.path.join(data_dir,"PRCF_reservation.csv")
data = pd.read_csv(path)
df= pd.DataFrame(data)

C:\Users\ingde\AppData\Local\Temp\ipykernel_23600\3903436939.py:4: DtypeWarning: Columns (7,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [80]:
df.sample(3)

,row_id,facility_name,facility_type,center_name,center_address_1,center_city,center_state,center_zip_code,center_country,center_longitude,center_latitude,center_geolocation,center_phone_number_1,center_fax,site_name,permit_number,permit_date,permit_year,permit_month,permit_status,event_type,event_start_date,event_start_year,event_start_month,event_start_time,event_end_date,event_end_year,event_end_month,event_end_time,day_of_week,attendance,hours_reserved,schedule_type,transaction_site,residency_flag,customer_gender,customer_city,customer_state,customer_zip_code,center_address_2,center_phone_number_2
450032,7652023-02-08 15:27:49.02023-07-0411:30:0012:0...,Rhodes - Zero Depth Pool,Pool,Rhodes Aquatic Complex,1860 S Longmore,Mesa,AZ,85202,US,-111.870846,33.380157,"{'type': 'Point', 'coordinates': [-111.8708460...",4.806443e+09,NaN,Pools/Aquatics,NaN,2023-02-08T15:27:49.000,2023,February,NaN,NaN,2023-07-04T00:00:00.000,2023,July,11:30:00,2023-07-04T00:00:00.000,2023,July,12:00:00,Tuesday,6.0,0.50,Automatic: Standard Activity,Pools/Aquatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5476,56553082022-02-1517:00:0021:00:00,Red Mountain Soccer Field 5,Field - Soccer - Complex,Red Mountain Soccer Complex,905 N Sunvalley Blvd,Mesa,AZ,85207,US,-111.672253,33.431074,"{'type': 'Point', 'coordinates': [-111.6722528...",4.806442e+09,NaN,Sports Complexes,55308.0,2021-11-23T14:31:07.000,2021,November,Approved,Community Youth Group Field,2022-02-15T00:00:00.000,2022,February,17:00:00,2022-02-15T00:00:00.000,2022,February,21:00:00,Tuesday,75.0,4.00,Reservation: Billable,Sports Complexes,True,Female,Mesa,AZ,85207,NaN,NaN
323411,17201052017-03-1410:30:0011:15:00,Art Studio (ArtVille),Room - Activity,i.d.e.a. Museum,150 W Pepper Pl,Mesa,AZ,85201,US,-111.836099,33.416210,"{'type': 'Point', 'coordinates': [-111.8360994...",4.806444e+09,4.806442e+09,i.d.e.a. Museum,20105.0,2017-02-23T15:18:26.000,2017,February,Approved,Group Bookings - Museums,2017-03-14T00:00:00.000,2017,March,10:30:00,2017-03-14T00:00:00.000,2017,March,11:15:00,Tuesday,8.0,0.75,Reservation: Billable,i.d.e.a. Museum,True,Female,Mesa,AZ,85201,NaN,4.806442e+09


To process efficiently the SQL connection, we need to drop the columns that do not add value:

In [81]:
df = df.drop(columns=df[['row_id', 'facility_type', 'center_name',
       'center_address_1', 'center_city', 'center_state', 'center_zip_code',
       'center_country', 'center_longitude', 'center_latitude',
       'center_geolocation', 'center_phone_number_1', 'center_fax',
       'permit_number', 'permit_date', 'permit_year',
       'permit_month', 'permit_status', 'event_type', 'event_start_date',
       'customer_city', 'customer_state', 'customer_zip_code',
       'center_address_2', 'center_phone_number_2']])

### SQL CONNECTION

Let's create a SQL connection, considering that Sqlite won't process over 10000 records:

In [82]:
con= sqlite3.connect('PRCF_data.db')

In [83]:
chunk_size = 10000
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    if i == 0:
        chunk.to_sql('PRCF_DATA', con, if_exists='replace', index=False, method="multi")
    else:
        chunk.to_sql('PRCF_DATA', con, if_exists='append', index=False, method="multi")

In [84]:
%load_ext sql
%sql sqlite:///PRCF_data.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


The connection has been successful.

### DATA REVIEW

Let's see the data collected:

In [85]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///PRCF_data.db
Done.


name
PRCF_DATA


In [86]:
%%sql 
SELECT * 
FROM PRCF_Data 
LIMIT 5

 * sqlite:///PRCF_data.db
Done.


facility_name,site_name,event_start_year,event_start_month,event_start_time,event_end_date,event_end_year,event_end_month,event_end_time,day_of_week,attendance,hours_reserved,schedule_type,transaction_site,residency_flag,customer_gender
Group Fitness,Recreation Centers,2022,January,10:00:00,2022-01-27T00:00:00.000,2022,January,11:00:00,Thursday,1.0,1.0,Reservation: Internal,Recreation Centers,0,Mixed
Group Fitness,Recreation Centers,2022,January,11:00:00,2022-01-27T00:00:00.000,2022,January,11:45:00,Thursday,1.0,0.75,Reservation: Internal,Recreation Centers,0,Mixed
Group Fitness,Recreation Centers,2022,January,12:00:00,2022-01-27T00:00:00.000,2022,January,12:45:00,Thursday,1.0,0.75,Reservation: Internal,Recreation Centers,0,Mixed
Group Fitness,Recreation Centers,2022,February,09:30:00,2022-02-15T00:00:00.000,2022,February,10:00:00,Tuesday,30.0,0.5,Reservation: Internal,Recreation Centers,0,Mixed
Gymnasium - Entire,Recreation Centers,2022,January,11:00:00,2022-01-27T00:00:00.000,2022,January,12:00:00,Thursday,1.0,1.0,Reservation: Internal,Recreation Centers,0,Mixed


In [87]:
%%sql
SELECT COUNT(*)
FROM PRCF_Data

 * sqlite:///PRCF_data.db
Done.


COUNT(*)
495000


We have to know the data types of each column for upcoming analysis:

In [88]:
cursor= con.cursor()
columnas = cursor.execute("PRAGMA table_info(PRCF_data)")

for columna in columnas:
    print(columna)

(0, 'facility_name', 'TEXT', 0, None, 0)
(1, 'site_name', 'TEXT', 0, None, 0)
(2, 'event_start_year', 'INTEGER', 0, None, 0)
(3, 'event_start_month', 'TEXT', 0, None, 0)
(4, 'event_start_time', 'TEXT', 0, None, 0)
(5, 'event_end_date', 'TEXT', 0, None, 0)
(6, 'event_end_year', 'INTEGER', 0, None, 0)
(7, 'event_end_month', 'TEXT', 0, None, 0)
(8, 'event_end_time', 'TEXT', 0, None, 0)
(9, 'day_of_week', 'TEXT', 0, None, 0)
(10, 'attendance', 'REAL', 0, None, 0)
(11, 'hours_reserved', 'REAL', 0, None, 0)
(12, 'schedule_type', 'TEXT', 0, None, 0)
(13, 'transaction_site', 'TEXT', 0, None, 0)
(14, 'residency_flag', 'INTEGER', 0, None, 0)
(15, 'customer_gender', 'TEXT', 0, None, 0)


The data agree with the information used previously.

### DATA SQL ANALYSIS

Let's see the data through SQL queries to find some business insigths:

#### FACILITY COUNTING

In [89]:
%%sql
SELECT site_name, COUNT(site_name) AS Count_site_name,transaction_site
FROM PRCF_Data
GROUP BY site_name
ORDER BY Count_site_name DESC


 * sqlite:///PRCF_data.db
Done.


site_name,Count_site_name,transaction_site
Mesa Tennis & Pickleball Center,148937,Mesa Tennis & Pickleball Center
Pools/Aquatics,97855,City of Mesa
Sports Complexes,83168,Sports Complexes
Park Sites,81331,City of Mesa
Recreation Centers,63223,Recreation Centers
Schools,15207,City of Mesa
Arizona Museum of Natural History,2634,Arizona Museum of Natural History
City of Mesa,1385,City of Mesa
i.d.e.a. Museum,1007,i.d.e.a. Museum
County Retail,158,City of Mesa


Easily we can see the quantity of centers dedicated to specific activities, where Tennis and other sports have the majority.

#### TOP TEN FACILITIES CENTERS 

In [90]:
%%sql
SELECT facility_name, COUNT(facility_name) AS Count_facility_name, transaction_site
FROM PRCF_Data
GROUP BY facility_name
ORDER BY Count_facility_name DESC
LIMIT 10

 * sqlite:///PRCF_data.db
Done.


facility_name,Count_facility_name,transaction_site
Group Fitness,9512,Recreation Centers
Pickleball Court 01,8113,Mesa Tennis & Pickleball Center
Tennis Court 01 (MTPC),7992,Mesa Tennis & Pickleball Center
Tennis Court 05 (MTPC),6949,Mesa Tennis & Pickleball Center
Tennis Court 02 (MTPC),6612,Mesa Tennis & Pickleball Center
Tennis Court 03 (MTPC),6309,Mesa Tennis & Pickleball Center
Tennis Court 13 (MTPC),6299,Mesa Tennis & Pickleball Center
Tennis Court 04 (MTPC),6049,Mesa Tennis & Pickleball Center
Tennis Court 06 (MTPC),5982,Mesa Tennis & Pickleball Center
Tennis Court 10 (MTPC),5819,Mesa Tennis & Pickleball Center


Although Mesa Tennis & Pickleball Center has the most centers adding them up, Group Fitness has the most centers individually.

#### EVENT START TIME PER DAY OF THE WEEK

In [91]:
%%sql
SELECT event_start_time, COUNT(event_start_time) AS Count_event_start_time, day_of_week, site_name
FROM PRCF_Data
GROUP BY event_start_time
HAVING Count_event_start_time > 2
ORDER BY day_of_week ASC
LIMIT 20

 * sqlite:///PRCF_data.db
Done.


event_start_time,Count_event_start_time,day_of_week,site_name
04:00:00,216,Friday,Pools/Aquatics
07:20:00,13,Friday,Pools/Aquatics
09:35:00,26,Friday,Recreation Centers
11:45:00,118,Friday,Recreation Centers
13:05:00,50,Friday,Mesa Tennis & Pickleball Center
18:00:00,40455,Friday,Mesa Tennis & Pickleball Center
19:00:00,12565,Friday,Mesa Tennis & Pickleball Center
21:00:00,10674,Friday,Mesa Tennis & Pickleball Center
21:30:00,5600,Friday,Mesa Tennis & Pickleball Center
00:00:00,41,Monday,Sports Complexes


It's rare taht Monday and Friday have events who started around at 4:00 - 5:00 hours, mostly around 5 a.m., they are days with different behaivor, but if we see the site_name columns we noticed that it's refer to pool events. In accordance with the day, Monday has a peak at 8:30, 9:00, and 13:00, it's means that the public preffer play tennis or pickball starting the week, and Friday public preffer play same sports but at night to close the week and starts the weekend.



#### EVENT END TIME AND DAY OF THE WEEK

In [92]:
%%sql
SELECT event_end_time, COUNT(event_end_time) AS Count_event_end_time, day_of_week
FROM PRCF_Data
GROUP BY event_end_time
HAVING Count_event_end_time> 2
ORDER BY day_of_week DESC
LIMIT 20

 * sqlite:///PRCF_data.db
Done.


event_end_time,Count_event_end_time,day_of_week
23:00:00,26679,Wednesday
22:00:00,89859,Wednesday
20:20:00,3,Wednesday
20:00:00,24291,Wednesday
19:45:00,691,Wednesday
19:40:00,68,Wednesday
19:20:00,1268,Wednesday
19:10:00,8,Wednesday
18:50:00,406,Wednesday
18:35:00,65,Wednesday


It's clear that Wednesday has the latest event end time. Let's see the top 10 latest hours:

In [93]:
%%sql
SELECT event_end_time, COUNT(event_end_time) AS Count_event_end_time, day_of_week
FROM PRCF_Data
GROUP BY event_end_time
HAVING Count_event_end_time> 2
ORDER BY event_end_time DESC
LIMIT 10

 * sqlite:///PRCF_data.db
Done.


event_end_time,Count_event_end_time,day_of_week
23:59:00,76,Saturday
23:45:00,20,Saturday
23:30:00,12,Saturday
23:00:00,26679,Wednesday
22:45:00,111,Friday
22:30:00,803,Saturday
22:15:00,51,Thursday
22:00:00,89859,Wednesday
21:59:00,20,Friday
21:45:00,755,Friday


Friday and Saturday are the most common latest event end time, with Saturday having the latest ending time.

#### TOP 30 EVENT TIME PER DAY OF THE WEEK

In [94]:
%%sql
SELECT site_name, event_start_time, event_end_time,
CAST(event_start_time AS TIME) AS start_time,
CAST(event_end_time AS TIME) AS end_time,

CASE
WHEN (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME)) >= 0 
THEN (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME))
ELSE (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME)) + 24
END AS event_time,

COUNT(event_start_time) AS Count_event, day_of_week, transaction_site
FROM PRCF_Data
GROUP BY event_start_time, event_end_time
ORDER BY event_time DESC
LIMIT 30

 * sqlite:///PRCF_data.db
Done.


site_name,event_start_time,event_end_time,start_time,end_time,event_time,Count_event,day_of_week,transaction_site
Recreation Centers,00:00:00,23:30:00,0,23,23,1,Saturday,Recreation Centers
Recreation Centers,00:00:00,23:45:00,0,23,23,1,Saturday,Recreation Centers
Pools/Aquatics,15:30:00,11:00:00,15,11,20,1,Monday,City of Mesa
Sports Complexes,04:00:00,23:00:00,4,23,19,178,Saturday,Recreation Centers
Recreation Centers,05:00:00,00:00:00,5,0,19,2,Friday,Recreation Centers
Pools/Aquatics,20:15:00,15:15:00,20,15,19,5,Friday,City of Mesa
Pools/Aquatics,04:00:00,22:00:00,4,22,18,24,Friday,City of Mesa
Pools/Aquatics,04:45:00,22:00:00,4,22,18,12,Saturday,City of Mesa
Sports Complexes,05:00:00,23:00:00,5,23,18,285,Thursday,Sports Complexes
Recreation Centers,05:00:00,23:59:00,5,23,18,3,Friday,Recreation Centers


We noticed that Recreation Centers have 23-hour reservations, normally on Saturdays. Curiously, there are reservations starting from 00:00 hours, as if there were people around the facilities at that hour. Normally, the events should start from 4:00–7:00 to 22:00–23:00; it's most probable that people start using the installations at those hours (sports mostly).

#### EVENT TIME WITHOUT OCCUPATION

In [98]:
%%sql

SELECT site_name, event_start_time, event_end_time, event_time, hours_reserved_time, (hours_reserved_time - event_time) AS time_without_occupation, 
        (event_time/hours_reserved_time*100) AS occupation_percentaje, schedule_type


FROM (

        SELECT site_name, event_start_time, event_end_time, schedule_type,

            CAST(event_start_time AS TIME) AS start_time,
            CAST(event_end_time AS TIME) AS end_time,
            CAST(hours_reserved AS TIME) AS hours_reserved_time,

            CASE

                WHEN (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME)) >= 0 
                THEN (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME))
                ELSE (CAST(event_end_time AS TIME) - CAST(event_start_time AS TIME)) + 24
            
            END AS event_time

        FROM PRCF_Data

    ) AS subquery
ORDER BY hours_reserved_time DESC
LIMIT 20

 * sqlite:///PRCF_data.db
Done.


site_name,event_start_time,event_end_time,event_time,hours_reserved_time,time_without_occupation,occupation_percentaje,schedule_type
Sports Complexes,00:00:00,00:00:00,0,96.0,96.0,0.0,Reservation: Maintenance
Sports Complexes,00:00:00,00:00:00,0,96.0,96.0,0.0,Reservation: Maintenance
Sports Complexes,00:00:00,00:00:00,0,96.0,96.0,0.0,Reservation: Maintenance
Stadiums,16:00:00,22:00:00,6,78.0,72.0,7.6923076923076925,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal
Recreation Centers,18:00:00,21:00:00,3,75.0,72.0,4.0,Reservation: Internal


There are 3-4 day internal reservations, but they were used less than 8% of the time; they could be reserved for fewer hours. There are a few 24-hour billable reservations with a maximum occupancy of 48.42%; they could be reserved for 1 day.